**9.	Which player has won the most tournaments in a single month?**


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

Reading requierd tables

In [4]:
MatchTournamentInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchTournamentInfo.csv')
MatchEventInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchEventInfo.csv')
MatchHomeTeamInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchHomeTeamInfo.csv')
MatchAwayTeamInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchAwayTeamInfo.csv')

Drop duplicated data

Creating a new table which contains match id, winer name and id and tournament name

In [59]:
merged_HW = MatchHomeTeamInfo_dd[['match_id' ,
                                  'player_id']].merge(MatchAwayTeamInfo_dd[['match_id' ,
                                                                            'player_id']],
                                                                             on ='match_id').drop_duplicates(subset='match_id')
merged_EHW = merged_HW[['match_id' ,
                        'player_id_x' , 'player_id_y']].merge(MatchEventInfo_dd[['match_id',
                                                                                 'winner_code']],
                                                                                  on ='match_id').drop_duplicates(subset='match_id')
merged_TEHW = merged_EHW[['match_id' ,
                        'player_id_x' , 'player_id_y' , 'winner_code']].merge(MatchTournamentInfo_dd[['match_id',
                                                                                 'tournament_name' , 'tournament_id']],
                                                                                  on ='match_id').drop_duplicates(subset='match_id')
merged_TEHW = merged_TEHW.dropna()
merged_TEHW['winner_id'] = merged_TEHW.apply( lambda x : x['player_id_x']  if x['winner_code']==1 else x['player_id_y'] , axis = 1)


,match_id,player_id_x,player_id_y,winner_code,tournament_name,tournament_id
0,12260075,14486,254742,2.0,"Madrid, Spain",129164
1,12260076,206570,90080,1.0,"Madrid, Spain",129164
2,12260077,46391,275923,2.0,"Madrid, Spain",129164
3,12260078,106755,275923,1.0,"Madrid, Spain",129164
4,12260080,221012,57163,1.0,"Madrid, Spain",129164


In [39]:
merged_TEHW.head()

,match_id,player_id_x,player_id_y,winner_code,tournament_name,tournament_id,winner_id
0,12260075,14486,254742,2.0,"Madrid, Spain",129164,254742
1,12260076,206570,90080,1.0,"Madrid, Spain",129164,206570
2,12260077,46391,275923,2.0,"Madrid, Spain",129164,275923
3,12260078,106755,275923,1.0,"Madrid, Spain",129164,106755
4,12260080,221012,57163,1.0,"Madrid, Spain",129164,221012


Creating a data frame to show the number of won tournoments for each player

In [ ]:
player = pd.concat ([MatchHomeTeamInfo[['player_id', 'name']] , MatchAwayTeamInfo[['player_id', 'name']]]).drop_duplicates(subset = 'player_id')
player['no_tour'] = 0
for no in player['player_id']:
    player['no_tour'].loc[player['player_id'] == no] = len(merged_TEHW['tournament_name'].loc[merged_TEHW['winner_id'] == no].unique())

In [55]:
player.head()

,player_id,name,no_tour
0,14486,Nadal R.,1
1,206570,Sinner J.,2
2,46391,Struff J.,1
3,106755,Rublev A.,2
4,221012,Cerundolo F.,2


**Reporting the best player with the most wins**

In [58]:
best_no_tour = player['no_tour'].max()
best_winer_id = player['player_id'].loc[player['no_tour'] == player['no_tour'].max()].values[0]
best_winer_name = player['name'].loc[player['no_tour'] == player['no_tour'].max()].values[0]
print (f'Player {best_winer_name} with {best_winer_id} ID has won {best_no_tour} tournomernt which is most tournaments in a single month')


Player Kuramochi M. with 390973 ID has won 7 tournomernt which is most tournaments in a single month
